In [1]:
FN_MODEL_OUTPUT = {'MLP':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy',
                   'RPN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy',
                   'CNN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy',
                   # 'cVAE': './model_outputs/cvae_preds_bestcrps.h5',
                   'cVAE': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/cvae.h5',
                   'HSR': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/hsr_preds_bestcrps.h5',
                  }

In [2]:
# model name
# (model name is used for the output)
model_name = 'MLP'

# input of validation dataset (npy)
fn_x_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_input_stride6.npy'

# true output of validation dataset (npy)
fn_y_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_target_stride6.npy'

# Model predicted output of varlidation dataset (npy)
fn_y_pred = FN_MODEL_OUTPUT[model_name]

# model grid information (nc)
fn_grid = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/grid_info/E3SM-MMF_ne4_grid-info.orig.nc'

# normalization scale factors (nc)
fn_mli_mean  = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_mean.nc'
fn_mli_min   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_min.nc'
fn_mli_max   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_max.nc'
fn_mlo_scale = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mlo_scale.nc'

# fn_save_output
fn_save_metrics = f'./metrics/{model_name}.metrics.csv'
fn_save_metrics_avg = f'./metrics/{model_name}.metrics.lev-avg.csv'

In [3]:
# physical constatns from (E3SM_ROOT/share/util/shr_const_mod.F90)
grav    = 9.80616    # acceleration of gravity ~ m/s^2
cp      = 1.00464e3  # specific heat of dry air   ~ J/kg/K
lv      = 2.501e6    # latent heat of evaporation ~ J/kg
lf      = 3.337e5    # latent heat of fusion      ~ J/kg
ls      = lv + lf    # latent heat of sublimation ~ J/kg
rho_air = 101325./ (6.02214e26*1.38065e-23/28.966) / 273.15 # density of dry air at STP  ~ kg/m^3
                                                            # ~ 1.2923182846924677
                                                            # SHR_CONST_PSTD/(SHR_CONST_RDAIR*SHR_CONST_TKFRZ)
                                                            # SHR_CONST_RDAIR   = SHR_CONST_RGAS/SHR_CONST_MWDAIR
                                                            # SHR_CONST_RGAS    = SHR_CONST_AVOGAD*SHR_CONST_BOLTZ
rho_h20 = 1.e3       # density of fresh water     ~ kg/m^ 3

vars_mlo_energy_conv = {'ptend_t':cp,
                        'ptend_q0001':lv,
                        'cam_out_NETSW':1.,
                        'cam_out_FLWDS':1.,
                        'cam_out_PRECSC':lv*rho_h20,
                        'cam_out_PRECC':lv*rho_h20,
                        'cam_out_SOLS':1.,
                        'cam_out_SOLL':1.,
                        'cam_out_SOLSD':1.,
                        'cam_out_SOLLD':1.
                       }

In [4]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

# set dimemsion names for xarray datasets
dim_name_level  = 'lev'
dim_name_sample = 'sample'

In [5]:
# load input dataset
x_true = np.load(fn_x_true).astype(np.float64)
y_true = np.load(fn_y_true).astype(np.float64)
if fn_y_pred[-3:] == '.h5':
    y_pred = xr.open_dataset(fn_y_pred)['pred'].values
else:
    y_pred = np.load(fn_y_pred).astype(np.float64)
N_samples = y_pred.shape[0] 

# load norm/scale factors
mlo_scale = xr.open_dataset(fn_mlo_scale)
mli_mean  = xr.open_dataset(fn_mli_mean)
mli_min   = xr.open_dataset(fn_mli_min)
mli_max   = xr.open_dataset(fn_mli_max)

In [6]:
# load grid information
ds_grid = xr.open_dataset(fn_grid) # has ncol:384
N_ncol = len(ds_grid['ncol']) # length of ncol dimension (nlat * nlon)

# make area-weights
ds_grid['area_wgt'] = ds_grid['area'] / ds_grid['area'].mean('ncol')

# map ds_grid's ncol dimension -> the N_samples dimension of npy-loayd arrays (e.g., y_pred)
to_xarray = {'area_wgt': (dim_name_sample,np.tile(ds_grid['area_wgt'], int(N_samples/len(ds_grid['ncol'])))),
            }
to_xarray = xr.Dataset(to_xarray)

# add nsample-mapped grid variables back to ds_grid
ds_grid = xr.merge([ds_grid  [['P0', 'hyai', 'hyam','hybi','hybm','lat','lon','area']],
                    to_xarray[['area_wgt']]])

In [7]:
# list of ML output variables
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC',
            'cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD'] # mlo mean ML output.

# length of each variable
vars_mlo_len = {'ptend_t':60,
                'ptend_q0001':60,
                'cam_out_NETSW':1,
                'cam_out_FLWDS':1,
                'cam_out_PRECSC':1,
                'cam_out_PRECC':1,
                'cam_out_SOLS':1,
                'cam_out_SOLL':1,
                'cam_out_SOLSD':1,
                'cam_out_SOLLD':1
               }

# map the length of dimension to the name of dimension
len_to_dim = {60:dim_name_level,
              N_samples: dim_name_sample}

In [8]:
# Here, we first construct a dictionary of {var name: (dimension name, array-like)},
# then, map the dictionary to an Xarray Dataset.
# (ref: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html)

DS = {}

for kds in ['true', 'pred']:
    if kds=='true':
        work = y_true
    elif kds=='pred':
        work = y_pred

    # [1] Construct dictionary for xarray dataset
    #     format is key for variable name /
    #               value for a turple of (dimension names, data).
    to_xarray = {}
    for k, kvar in enumerate(vars_mlo):

        # length of variable (ie, number of levels)
        kvar_len = vars_mlo_len[kvar]

        # set dimensions of variable
        if kvar_len == 60:
            kvar_dims = (dim_name_sample, dim_name_level)
        elif kvar_len == 1:
            kvar_dims = dim_name_sample

        # set start and end indices of variable in the loaded numpy array
        # then, add 'kvar':(kvar_dims, <np_array>) to dictionary
        if k==0: ind1=0
        ind2 = ind1 + kvar_len

        # scaled output
        kvar_data = np.squeeze(work[:,ind1:ind2])
        # unscaled output
        kvar_data = kvar_data / mlo_scale[kvar].values

        to_xarray[kvar] = (kvar_dims, kvar_data)

        ind1 = ind2

    # [2] convert dict to xarray dataset
    DS[kds] = xr.Dataset(to_xarray)

    # [3] add surface pressure ('state_ps') from ml input
    # normalized ps
    state_ps =  xr.DataArray(x_true[:,120], dims=('sample'), name='state_ps')
    # denormalized ps
    state_ps = state_ps * (mli_max['state_ps'] - mli_min['state_ps']) + mli_mean['state_ps']
    DS[kds]['state_ps'] = state_ps

    # [4] add grid information
    DS[kds] = xr.merge([DS[kds], ds_grid])

    # [5] add pressure thickness of each level, dp
    # FYI, in a hybrid sigma vertical coordinate system, pressure at level z is
    # P[x,z] = hyam[z]*P0 + hybm[z]*PS[x,z],
    # where, hyam and hybm are 
    tmp = DS[kds]['P0']*DS[kds]['hyai'] + DS[kds]['state_ps']*DS[kds]['hybi']
    tmp = tmp.isel(ilev=slice(1,61)).values - tmp.isel(ilev=slice(0,60)).values
    tmp = tmp.transpose()
    DS[kds]['dp'] = xr.DataArray(tmp, dims=('sample', 'lev'))

    # [6] break (sample) to (ncol,time)
    N_timestep = int(N_samples/N_ncol)
    dim_ncol     = np.arange(N_ncol)
    dim_timestep = np.arange(N_timestep)
    new_ind = pd.MultiIndex.from_product([dim_timestep, dim_ncol],
                                         names=['time', 'ncol'])
    DS[kds] = DS[kds].assign_coords(sample=new_ind).unstack('sample')

In [52]:
# [1] Weight vertical levels by dp/g that is equivalent to a mass of air within a grid cell per unit area [kg/m2]
# [2] Weight horizontal area of each grid cell by a[x]/mean(a[x]).
# [3] Unit conversion to a common energy unit

DS_ENERGY = {}
for kds in ['true','pred']:
    # Make a copy to keep original dataset
    DS_ENERGY[kds] = DS[kds].copy(deep=True)

    # vertical weighting / area weighting / unit conversion
    for kvar in vars_mlo:

        # [1] weight vertical levels by dp/g
        #     ONLY for vertically-resolved variables, e.g., ptend_{t,q0001}
        # dp/g = - \rho * dz
        if vars_mlo_len[kvar] == 60:
            DS_ENERGY[kds][kvar] = DS_ENERGY[kds][kvar] * DS_ENERGY[kds]['dp']/grav

        # [2] weight area
        #     for ALL variables
        DS_ENERGY[kds][kvar] = DS_ENERGY[kds]['area_wgt'] * DS_ENERGY[kds][kvar]

        # [3] convert units to W/m2
        #     for variables with different units, e.g., ptend_{t,q0001}, precsc, precc
        DS_ENERGY[kds][kvar] =  vars_mlo_energy_conv[kvar] * DS_ENERGY[kds][kvar]

In [113]:
true_ex = np.array(DS_ENERGY['true'][vars_mlo[0]])
true_ex = true_ex.transpose((2, 0, 1))
true_ex.shape

(60, 4380, 384)

In [114]:
hakd = y_true[:, 0:60].reshape((4380, 384, 60)).transpose((2,0,1))
hakd.shape

(60, 4380, 384)

In [115]:
true_ex - hakd

array([[[ 8.41118637e-03,  8.88525512e-03,  6.82550445e-03, ...,
         -1.51547235e-02, -2.02621738e-02, -2.15350918e-02],
        [ 8.96014070e-03,  9.84809855e-03,  6.49007921e-03, ...,
         -1.85679729e-02, -2.11594218e-02, -2.44147648e-02],
        [ 8.52260162e-03,  9.83346078e-03,  6.49539277e-03, ...,
         -1.87976515e-02, -2.02977505e-02, -2.36408155e-02],
        ...,
        [-2.44532201e-02, -2.41002679e-02, -2.22537344e-02, ...,
          5.99483523e-03,  6.42485327e-03,  7.00597670e-03],
        [-1.94589131e-02, -1.65689751e-02, -1.63493809e-02, ...,
          1.04144947e-02,  6.68652422e-03,  6.63834607e-03],
        [ 1.05771755e-02,  7.61777407e-03,  1.12765876e-02, ...,
          1.54444472e-02, -1.83849227e-02, -1.26470046e-02]],

       [[ 8.71206602e-03,  4.09028666e-03,  1.16242282e-02, ...,
         -2.59038854e-03,  1.32702160e-03,  1.42714086e-03],
        [ 7.66089423e-03,  3.40794590e-03,  1.16246078e-02, ...,
         -4.32521171e-03,  1.11812594e

In [109]:
y_pred[:, 0:60].reshape((4380, 384, 60)).transpose((2,0,1)).shape

(60, 4380, 384)

In [87]:
pred_ex = np.array(DS_ENERGY['pred'][vars_mlo[2]])
pred_ex.shape

(4380, 384)

In [90]:
wowza = y_pred[:,120:].reshape((4380, 384, -1)).transpose((2, 0, 1))
wowza.shape

(8, 4380, 384)

In [98]:
wak = y_pred[:,120]

In [97]:
hak = pred_ex

In [105]:
wak[0:70]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00191455, 0.15175636])

In [106]:
hak[0,:][0:70]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.71206117, 58.22

In [91]:
wowza[0,:,:] - pred_ex

array([[   0.        ,    0.        ,    0.        , ...,  -40.43053667,
        -251.99332857, -207.65119849],
       [   0.        ,    0.        ,    0.        , ..., -146.21779391,
        -328.37488336, -341.46216983],
       [   0.        ,    0.        ,    0.        , ..., -172.14034025,
        -258.14432347, -320.97759941],
       ...,
       [-461.12415566, -292.78258824, -391.92978743, ...,    0.        ,
           0.        ,    0.        ],
       [-130.40535454,  -16.48287417,  -69.0741107 , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
         -59.65105643,  -15.66914151]])

In [57]:
DS_ENERGY['pred']['dp'].shape

(60, 4380, 384)

In [64]:
DS_ENERGY['pred']['area_wgt'].shape

(4380, 384)

In [68]:
var_num = 2
np.array(DS_ENERGY['pred'][vars_mlo[var_num]]).shape

(4380, 384)

In [69]:
y_pred.shape

(1681920, 128)

In [71]:
DS_ENERGY['pred']['area_wgt'].shape

(4380, 384)

In [10]:
state_ps.shape

(1681920,)

In [11]:
x_true[:,120].shape

(1681920,)

In [12]:
mli_max['state_ps']

<xarray.DataArray 'state_ps' ()>
array(105050.071231)

In [13]:
mli_min['state_ps']

<xarray.DataArray 'state_ps' ()>
array(69598.899331)

In [14]:
mli_mean['state_ps']

<xarray.DataArray 'state_ps' ()>
array(98623.665213)

In [15]:
state_ps_alt = x_true[:,120]*(mli_max['state_ps'].values - mli_min['state_ps'].values) + mli_mean['state_ps'].values
state_ps_alt_reshaped = np.reshape(state_ps_alt, (-1,384))

In [16]:
state_ps_alt.shape

(1681920,)

In [17]:
state_ps_alt_reshaped.shape

(4380, 384)

In [18]:
hmm2 = np.array(ds_grid['P0']*ds_grid['hyai'])[:,np.newaxis,np.newaxis] + ds_grid['hybi'].values[:, np.newaxis, np.newaxis] * state_ps_alt_reshaped[np.newaxis, :, :]

In [19]:
hmm2.shape

(61, 4380, 384)

In [20]:
lol = hmm2[1:61,:,:] - hmm2[0:60,:,:]

In [21]:
dp_ref = DS[kds]['dp'].values

In [22]:
np.sum(lol-dp_ref)

0.0

In [23]:
dp_ref.shape

(60, 4380, 384)

In [25]:
ds_grid['area_wgt'].shape

(1681920,)

In [29]:
DS['true']['area_wgt']

<xarray.DataArray 'area_wgt' (time: 4380, ncol: 384)>
array([[0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       ...,
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 4373 4374 4375 4376 4377 4378 4379
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383

In [50]:
okeh = y_pred[:, 0:60].shape

In [51]:
okeh

(1681920, 60)

In [45]:
DS['true']['dp']

<xarray.DataArray 'dp' (lev: 60, time: 4380, ncol: 384)>
array([[[   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        ...,
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817],
        [   4.49334817,    4.49334817,    4.49334817, ...,
            4.49334817,    4.49334817,    4.49334817]],

       [[   8.05875326,    8.05875326,    8.05875326, ...,
            8.05875326,    8.05875326,    8.05875326],
        [   8.05875326,    8.05875326,    8.05875326, ...,
            8.05875326,    8.05875326,    8.05875326],
        [   8.05875326,    8.05875326,    8.05875326, ...,
            8.05875326,    8.05875326,    8.05875326],
...
        [1459.9266813 , 1471.22505066, 1430.42324241, ...,
         1385.27574414, 1465.39475656, 1464.28781425],
        [1458.73305834, 1470.35831927, 1429.63945825, ...,
         1385.80204525, 1466.67312371, 1465.17957177],
        [1459.75239868, 1471.00650404, 1431.74712721, ...,
         1387.53251194, 1467.96758963, 1467.26054248]],

       [[1489.97206792, 1502.89740105, 1458.63716644, ...,
         1384.42388415, 1458.37787713, 1460.30758745],
        [1489.4934162 , 1501.23566766, 1458.43904255, ...,
         1384.08881605, 1457.67663963, 1459.64745319],
        [1487.3687768 , 1500.10258501, 1455.3836193 , ...,
         1382.80318621, 1457.16745813, 1458.11426899],
        ...,
        [1481.41055125, 1492.87518424, 1451.47294806, ...,
         1405.66107192, 1486.95909317, 1485.83586141],
        [1480.19936327, 1491.99569827, 1450.67762996, ...,
         1406.19511793, 1488.25627241, 1486.74074177],
        [1481.23370393, 1492.65342155, 1452.81631478, ...,
         1407.95104968, 1489.56978733, 1488.85233547]]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 4373 4374 4375 4376 4377 4378 4379
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383
  * lev      (lev) float64 0.07835 0.1411 0.2529 0.4493 ... 963.6 978.0 992.6

In [30]:
DS['pred']['area_wgt']

<xarray.DataArray 'area_wgt' (time: 4380, ncol: 384)>
array([[0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       ...,
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478],
       [0.83877478, 0.9382394 , 0.9382394 , ..., 0.9382394 , 0.9382394 ,
        0.83877478]])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 4373 4374 4375 4376 4377 4378 4379
  * ncol     (ncol) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383

In [34]:
hola = np.array(ds_grid['area_wgt'])
hola.shape

(1681920,)

In [35]:
adios = np.reshape(hola, (-1, 384))

In [36]:
adios.shape

(4380, 384)